<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark/challenges/challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession.builder.master('local').appName('Cleansing').getOrCreate()

/Users/ineslopes/Documents/de_edit/.pipelines/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
24/12/01 23:21:02 WARN Utils: Your hostname, Iness-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
24/12/01 23:21:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/01 23:21:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
lines_result=spark.read.format("parquet").load("./content/lake/bronze/lines/*")
vehicles_result = spark.read.format("parquet").load("./content/lake/bronze/vehicles/*")
municipalities_result = spark.read.format("parquet").load("./content/lake/bronze/municipalities/*")

In [5]:
#Apply basic transform
df_v=(vehicles_result
      .withColumnsRenamed({'lat':'latitude','lon':'longitude'})
      .dropDuplicates()
      .dropna(subset='CURRENT_STATUS')
      .withColumn("date", expr("date(timestamp)"))
      )

df_l=(lines_result
      .dropDuplicates()
      .dropna(subset='LONG_NAME')
      )

df_m=(municipalities_result
      .dropDuplicates()
      .dropna(how='any',subset=['DISTRICT_NAME','NAME'])
      )

In [10]:
(df_v.coalesce(1)
 .write.mode("overwrite")
 .partitionBy("date")
 .format("parquet")
 .save("./content/lake/silver/vehicles")
)
 
(df_l.coalesce(1)
 .write.mode("overwrite")
 .format("parquet")
 .save("./content/lake/silver/lines")
)
 
(df_m.coalesce(1)
 .write.mode("overwrite")
 .format("parquet")
 .save("./content/lake/silver/municipalities")
)

# CHALLENGE 2
##  Implement CLEANSING process
- Set up path in the "lake"
  - !mkdir -p /content/lake/silver

- Read data from BRONZE layer as PARQUET:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities

- Transformations
  - vehicles
    - rename "lat" and "lon" to "latitude" and "longitude" respectively
    - remove possible duplicates
    - remove rows when the column CURRENT_STATUS is null
    - remove any corrupted record
  - lines
    - remove duplicates
    - remove rows when the column LONG_NAME is null
    - remove any corrupted record
  - municipalities
    - remove duplicates
    - remove rows when the columns NAME or DISTRICT_NAME are null
    - remove any corrupted record

- Write data as PARQUET into the SILVER layer (/content/lake/silver)
  - Partition "vehicles" by "date"(created in the ingestion)
  - Paths:
    - vehicles - path: /content/lake/silver/vehicles
    - lines - path: /content/lake/silver/lines
    - municipalities - path: /content/lake/silver/municipalities

# Setting up PySpark

In [ ]:
%pip install pyspark